# 🧪 Laboratorio 2: Criptografía Asimétrica — RSA

## 📚 Contenido
- **Parte 1**: Ejemplo Práctico Completo (Cifrar "ATTACK")
- **Parte 2**: Teoría y Fundamentos Matemáticos

## 🎯 Objetivos
- Implementar RSA completo desde cero
- Entender cada componente matemático
- Cifrar y descifrar mensajes de texto

---

# 🚀 PARTE 1: EJEMPLO PRÁCTICO

## Cifrado y Descifrado de "ATTACK"

### 📝 Configuración Inicial

**Modifica estos valores para experimentar:**

In [ ]:
# ===================================================================
# PARÁMETROS DEL SISTEMA RSA
# ===================================================================

# Primos (deben ser primos distintos)
p = 61
q = 53

# Exponente público (debe cumplir gcd(e, φ(n)) = 1)
e = 17

# Mensaje a cifrar (solo letras A-Z)
MENSAJE = "ATTACK"

# Calcular módulo n = p × q
n = p * q

print("="*70)
print("CONFIGURACIÓN DEL SISTEMA RSA")
print("="*70)
print(f"Primo p:            {p}")
print(f"Primo q:            {q}")
print(f"Módulo n = p×q:     {n}")
print(f"Exponente público:  {e}")
print(f"Mensaje a cifrar:   {MENSAJE}")
print("="*70)

---

## 1️⃣ Función: Máximo Común Divisor (MCD)

In [ ]:
def gcd(a: int, b: int) -> int:
    """
    Algoritmo de Euclides para calcular el MCD.
    
    Funcionamiento:
    - Divide a entre b y toma el residuo
    - Reemplaza a con b, y b con el residuo
    - Repite hasta que b sea 0
    - El último valor de a es el MCD
    
    Ejemplo: gcd(48, 18)
      48 = 18×2 + 12
      18 = 12×1 + 6
      12 = 6×2 + 0  → MCD = 6
    """
    while b != 0:
        a, b = b, a % b
    return abs(a)

print("✓ Función gcd() cargada")

---

## 2️⃣ Función: Euclides Extendido

In [ ]:
def extended_gcd(a: int, b: int) -> tuple:
    """
    Algoritmo de Euclides Extendido.
    Retorna (gcd, x, y) tal que: a×x + b×y = gcd(a,b)
    
    Funcionamiento:
    - Usa recursión para llegar al caso base (b=0)
    - Al regresar, calcula los coeficientes x, y
    - Estos coeficientes son la Identidad de Bézout
    
    Aplicación en RSA:
    - Se usa para calcular el inverso modular
    - Necesario para encontrar la clave privada d
    """
    # Caso base: si b=0, entonces gcd=a, x=1, y=0
    if b == 0:
        return abs(a), 1 if a >= 0 else -1, 0
    
    # Llamada recursiva
    gcd_val, x1, y1 = extended_gcd(b, a % b)
    
    # Calcular coeficientes actuales
    x = y1
    y = x1 - (a // b) * y1
    
    return gcd_val, x, y

print("✓ Función extended_gcd() cargada")

---

## 3️⃣ Función: Inverso Modular

In [ ]:
def modular_inverse(a: int, m: int) -> int:
    """
    Calcula el inverso modular de 'a' módulo 'm'.
    Encuentra x tal que: (a × x) mod m = 1
    
    Funcionamiento:
    1. Usa extended_gcd para obtener x tal que a×x + m×y = gcd
    2. Si gcd(a,m) = 1, entonces x es el inverso modular
    3. Aplica módulo m para asegurar resultado positivo
    
    Aplicación en RSA:
    - Se usa para calcular d = e^(-1) mod φ(n)
    - d es la clave privada para descifrar
    """
    gcd_val, x, _ = extended_gcd(a, m)
    
    # Verificar que exista inverso (solo si son coprimos)
    if gcd_val != 1:
        raise ValueError(f"No existe inverso: gcd({a}, {m}) = {gcd_val} ≠ 1")
    
    # Retornar inverso positivo
    return x % m

print("✓ Función modular_inverse() cargada")

---

## 4️⃣ Función: Exponenciación Modular Rápida

In [ ]:
def modular_exponentiation(base: int, exp: int, mod: int) -> int:
    """
    Calcula (base^exp) mod mod de forma eficiente.
    Algoritmo: Square-and-Multiply
    
    Funcionamiento:
    1. Convierte exponente a binario
    2. Por cada bit del exponente (de derecha a izquierda):
       - Si bit=1: multiplica result por base
       - Siempre: eleva base al cuadrado
    3. Aplica módulo en cada paso para mantener números pequeños
    
    Ejemplo: 5^13 mod 7
      13 en binario = 1101
      Procesa bits: 1,1,0,1
      Complejidad: O(log exp) en lugar de O(exp)
    
    Aplicación en RSA:
    - Cifrado: C = M^e mod n
    - Descifrado: M = C^d mod n
    - Sin este algoritmo, RSA sería impráctico
    """
    result = 1
    base = base % mod
    
    while exp > 0:
        # Si el bit actual es 1, multiplicar por base
        if exp & 1:  # exp & 1 verifica si el último bit es 1
            result = (result * base) % mod
        
        # Elevar base al cuadrado
        base = (base * base) % mod
        
        # Desplazar exponente un bit a la derecha (dividir entre 2)
        exp >>= 1
    
    return result

print("✓ Función modular_exponentiation() cargada")

---

## 5️⃣ Función: Texto a Números

In [ ]:
def text_to_numbers(text: str) -> str:
    """
    Convierte texto a representación numérica.
    Mapeo: A=00, B=01, C=02, ..., Z=25
    
    Funcionamiento:
    1. Convierte texto a mayúsculas
    2. Para cada letra, calcula su valor: ord(letra) - ord('A')
    3. Formatea como 2 dígitos con ceros a la izquierda
    4. Concatena todos los pares de dígitos
    
    Ejemplo:
      "HELLO" → "0704111114"
      H=07, E=04, L=11, L=11, O=14
    """
    text = text.upper()
    numeric_str = ""
    
    for char in text:
        if 'A' <= char <= 'Z':
            # Calcular posición en el alfabeto (A=0, B=1, ...)
            value = ord(char) - ord('A')
            # Agregar como 2 dígitos (ej: 7 → "07")
            numeric_str += f"{value:02d}"
    
    return numeric_str

print("✓ Función text_to_numbers() cargada")

---

## 6️⃣ Función: Números a Texto

In [ ]:
def numbers_to_text(numeric_str: str) -> str:
    """
    Convierte números de vuelta a texto.
    
    Funcionamiento:
    1. Procesa la cadena en pares de dígitos
    2. Convierte cada par a entero
    3. Suma ord('A') para obtener el código ASCII
    4. Convierte a caracter con chr()
    
    Ejemplo:
      "0704111114" → "HELLO"
      07→H, 04→E, 11→L, 11→L, 14→O
    """
    text = ""
    
    # Procesar de 2 en 2 dígitos
    for i in range(0, len(numeric_str), 2):
        if i + 1 < len(numeric_str):
            pair = numeric_str[i:i+2]
            value = int(pair)
            # Verificar que esté en rango válido (0-25)
            if 0 <= value <= 25:
                text += chr(ord('A') + value)
    
    return text

print("✓ Función numbers_to_text() cargada")

---

## 7️⃣ Función: Dividir en Bloques

In [ ]:
def split_into_blocks(numeric_str: str, modulus: int) -> list:
    """
    Divide la cadena numérica en bloques menores que el módulo n.
    Usa algoritmo greedy para maximizar el tamaño de cada bloque.
    
    Funcionamiento:
    1. Inicia con un bloque vacío
    2. Intenta agregar la siguiente letra (2 dígitos)
    3. Si el bloque resultante >= n, corta y empieza nuevo bloque
    4. Guarda (valor_bloque, cantidad_de_letras)
    
    ¿Por qué guardar la cantidad de letras?
    - Para reconstruir correctamente al descifrar
    - Permite agregar ceros a la izquierda (padding)
    - Ejemplo: 210 con 3 letras → "000210" → A,C,K
    
    Restricción RSA:
    - Cada bloque M debe cumplir: 0 ≤ M < n
    - Si M ≥ n, la operación M^e mod n no es reversible
    """
    blocks = []
    i = 0
    
    while i < len(numeric_str):
        current_block = ""
        letters_count = 0
        
        # Intentar agregar más letras al bloque actual
        while i < len(numeric_str):
            if i + 1 < len(numeric_str):
                next_pair = numeric_str[i:i+2]
                test_block = current_block + next_pair
                test_value = int(test_block)
                
                # Verificar si aún cabe en el módulo
                if test_value < modulus:
                    current_block = test_block
                    letters_count += 1
                    i += 2
                else:
                    # No cabe, cortar bloque aquí
                    break
            else:
                break
        
        # Agregar bloque si no está vacío
        if current_block:
            blocks.append((int(current_block), letters_count))
    
    return blocks

print("✓ Función split_into_blocks() cargada")

---

## 8️⃣ Calcular φ(n) y Clave Privada d

In [ ]:
# Calcular φ(n) usando la fórmula de Euler
# φ(n) = (p-1)(q-1) para n = p×q con p,q primos
phi = (p - 1) * (q - 1)

print("="*70)
print("CÁLCULO DE CLAVES")
print("="*70)
print(f"\n1. Función de Euler:")
print(f"   φ(n) = (p-1)(q-1) = ({p}-1)×({q}-1) = {p-1}×{q-1} = {phi}")

# Verificar que e y φ(n) sean coprimos
gcd_val = gcd(e, phi)
print(f"\n2. Verificación de e:")
print(f"   gcd(e, φ(n)) = gcd({e}, {phi}) = {gcd_val}")

if gcd_val != 1:
    print("\n   ❌ ERROR: e y φ(n) no son coprimos.")
    print("   Cambia el valor de e en la primera celda.")
else:
    print("   ✓ e y φ(n) son coprimos (condición cumplida)")
    
    # Calcular clave privada d
    # d es el inverso modular de e módulo φ(n)
    # Es decir: (e × d) mod φ(n) = 1
    d = modular_inverse(e, phi)
    
    print(f"\n3. Clave privada:")
    print(f"   d = inverso_modular({e}, {phi}) = {d}")
    
    # Verificar que d es correcto
    verification = (e * d) % phi
    print(f"\n4. Verificación:")
    print(f"   (e × d) mod φ(n) = ({e} × {d}) mod {phi} = {verification}")
    
    if verification == 1:
        print("   ✓ Clave privada calculada correctamente")
    
    print("\n" + "="*70)
    print("RESUMEN DE CLAVES")
    print("="*70)
    print(f"Clave Pública:   (e={e}, n={n})")
    print(f"Clave Privada:   (d={d}, n={n})")
    print(f"Secretos:        p={p}, q={q}, φ(n)={phi}")
    print("="*70)

---

## 9️⃣ PASO 1: Convertir Mensaje a Números

In [ ]:
print("="*70)
print("PASO 1: MAPEO DE TEXTO A NÚMEROS")
print("="*70)

print(f"\nMensaje original: {MENSAJE}")
print(f"\nConversión letra por letra:")

# Mostrar conversión de cada letra
for char in MENSAJE:
    value = ord(char) - ord('A')
    print(f"  {char} → {value:02d}")

# Obtener representación numérica completa
numeric_repr = text_to_numbers(MENSAJE)

print(f"\nCadena numérica completa: {numeric_repr}")
print(f"Longitud: {len(numeric_repr)} dígitos ({len(MENSAJE)} letras × 2)")
print("="*70)

---

## 🔟 PASO 2: Dividir en Bloques

In [ ]:
print("="*70)
print("PASO 2: DIVISIÓN EN BLOQUES")
print("="*70)

print(f"\nCadena numérica: {numeric_repr}")
print(f"Módulo n: {n}")
print(f"Restricción: Cada bloque debe ser < {n}\n")

# Dividir en bloques
blocks = split_into_blocks(numeric_repr, n)

print("Bloques generados (algoritmo greedy):\n")

for i, (block_value, num_letters) in enumerate(blocks, 1):
    # Reconstruir con padding para mostrar
    block_str = str(block_value).zfill(num_letters * 2)
    
    # Extraer letras del bloque
    letters = []
    for j in range(0, len(block_str), 2):
        pair = block_str[j:j+2]
        letter = chr(ord('A') + int(pair))
        letters.append(letter)
    
    letters_str = ''.join(letters)
    
    print(f"Bloque {i}:")
    print(f"  Valor entero:   {block_value}")
    print(f"  Con padding:    '{block_str}'")
    print(f"  Letras:         {letters_str} ({num_letters} letra(s))")
    print(f"  Verificación:   {block_value} < {n} ✓")
    print()

print("="*70)

---

## 1️⃣1️⃣ PASO 3: Cifrado (C = M^e mod n)

In [ ]:
print("="*70)
print("PASO 3: ENCRIPTACIÓN")
print("="*70)
print(f"\nFórmula: C = M^e mod n = M^{e} mod {n}\n")

encrypted_blocks = []

for i, (M, num_letters) in enumerate(blocks, 1):
    # Cifrar usando exponenciación modular rápida
    C = modular_exponentiation(M, e, n)
    encrypted_blocks.append((C, num_letters))
    
    print(f"Bloque {i}:")
    print(f"  M (texto plano) = {M}")
    print(f"  C (cifrado)     = {M}^{e} mod {n} = {C}")
    print()

# Lista solo con valores cifrados (sin metadatos)
cipher_list = [C for C, _ in encrypted_blocks]

print("="*70)
print(f"✓ MENSAJE CIFRADO: {cipher_list}")
print("="*70)
print("\nNota: Los bloques cifrados son NÚMEROS ENTEROS, no letras.")
print("      Solo el mensaje original y descifrado están en letras.")

---

## 1️⃣2️⃣ PASO 4: Descifrado (M = C^d mod n)

In [ ]:
print("="*70)
print("PASO 4: DESENCRIPTACIÓN")
print("="*70)
print(f"\nFórmula: M = C^d mod n = C^{d} mod {n}\n")

decrypted_blocks = []

for i, (C, num_letters) in enumerate(encrypted_blocks, 1):
    # Descifrar usando la clave privada d
    M = modular_exponentiation(C, d, n)
    decrypted_blocks.append((M, num_letters))
    
    print(f"Bloque {i}:")
    print(f"  C (cifrado)          = {C}")
    print(f"  M (texto recuperado) = {C}^{d} mod {n} = {M}")
    print()

print("="*70)

---

## 1️⃣3️⃣ PASO 5: Reconstrucción del Mensaje

In [ ]:
print("="*70)
print("PASO 5: RECONSTRUCCIÓN DEL MENSAJE")
print("="*70)
print()

reconstructed_numeric = ""

for i, (M, num_letters) in enumerate(decrypted_blocks, 1):
    # Calcular longitud esperada (2 dígitos por letra)
    expected_length = num_letters * 2
    
    # Agregar padding de ceros a la izquierda
    # Ejemplo: 210 con 3 letras → "000210"
    block_str = str(M).zfill(expected_length)
    
    print(f"Bloque {i}:")
    print(f"  Valor descifrado:  {M}")
    print(f"  Número de letras:  {num_letters}")
    print(f"  Longitud esperada: {num_letters} × 2 = {expected_length} dígitos")
    print(f"  Sin padding:       '{M}'")
    print(f"  Con padding:       '{block_str}'")
    print()
    
    # Desglose letra por letra
    print(f"  Desglose por letra:")
    for j in range(0, len(block_str), 2):
        pair = block_str[j:j+2]
        value = int(pair)
        letter = chr(ord('A') + value)
        print(f"    '{pair}' → {value:2d} → {letter}")
    print()
    
    # Concatenar a la cadena numérica total
    reconstructed_numeric += block_str

print("="*70)
print(f"Cadena numérica reconstruida: {reconstructed_numeric}")
print(f"Cadena numérica original:     {numeric_repr}")

if reconstructed_numeric == numeric_repr:
    print("\n✓ Las cadenas numéricas coinciden perfectamente ✓")
else:
    print("\n✗ ERROR: Las cadenas no coinciden")

print("="*70)

---

## 1️⃣4️⃣ RESULTADO FINAL

In [ ]:
# Convertir cadena numérica de vuelta a texto
decrypted_message = numbers_to_text(reconstructed_numeric)

print("="*70)
print("RESULTADO FINAL")
print("="*70)
print()
print(f"Mensaje original:       {MENSAJE}")
print(f"Bloques cifrados:       {cipher_list}")
print(f"Mensaje descifrado:     {decrypted_message}")
print()

if MENSAJE == decrypted_message:
    print("✓✓✓ ÉXITO COMPLETO ✓✓✓")
    print("El mensaje fue cifrado y descifrado correctamente.")
else:
    print("✗✗✗ ERROR ✗✗✗")
    print("El mensaje descifrado no coincide con el original.")

print()
print("="*70)
print("RESUMEN DEL SISTEMA RSA")
print("="*70)
print(f"Claves públicas:    (e={e}, n={n})")
print(f"Claves privadas:    (d={d}, n={n})")
print(f"Parámetros secretos: p={p}, q={q}, φ(n)={phi}")
print("="*70)

---

# 📚 PARTE 2: TEORÍA Y FUNDAMENTOS

## Pruebas Unitarias de Funciones

In [ ]:
print("="*70)
print("PRUEBAS DE FUNCIONES MATEMÁTICAS")
print("="*70)

print("\n1. Algoritmo de Euclides (MCD):")
print(f"   gcd(48, 18) = {gcd(48, 18)} (esperado: 6)")
print(f"   gcd(17, 3120) = {gcd(17, 3120)} (esperado: 1)")
print(f"   gcd(100, 35) = {gcd(100, 35)} (esperado: 5)")

print("\n2. Euclides Extendido:")
g, x, y = extended_gcd(17, 3120)
print(f"   extended_gcd(17, 3120) = ({g}, {x}, {y})")
print(f"   Verificación: 17×{x} + 3120×{y} = {17*x + 3120*y}")

print("\n3. Inverso Modular:")
inv = modular_inverse(17, 3120)
print(f"   modular_inverse(17, 3120) = {inv}")
print(f"   Verificación: (17 × {inv}) % 3120 = {(17 * inv) % 3120}")

print("\n4. Exponenciación Modular:")
result = modular_exponentiation(5, 13, 7)
print(f"   5^13 mod 7 = {result}")
print(f"   Verificación con pow(): {pow(5, 13, 7)}")
print(f"   ¿Coinciden? {result == pow(5, 13, 7)}")

---

## Visualización de Exponenciación Modular

In [ ]:
def modular_exponentiation_traced(base: int, exp: int, mod: int) -> int:
    """
    Versión educativa con trazabilidad completa.
    Muestra cada paso del algoritmo Square-and-Multiply.
    """
    print(f"\nCalculando {base}^{exp} mod {mod}")
    print("="*70)
    
    # Mostrar exponente en binario
    exp_binary = bin(exp)[2:]
    print(f"\nExponente en binario: {exp} = {exp_binary}")
    print(f"Procesando {len(exp_binary)} bits de izquierda a derecha\n")
    
    print(f"{'Paso':<6} {'Bit':<6} {'Operación':<40} {'Resultado':<10}")
    print("-"*70)
    
    result = 1
    base = base % mod
    temp_exp = exp
    step = 0
    
    # Obtener bits del exponente
    bits = []
    while temp_exp > 0:
        bits.append(temp_exp & 1)
        temp_exp >>= 1
    bits.reverse()
    
    # Primer bit (siempre 1)
    result = base
    print(f"{step+1:<6} {1:<6} result = base = {base}{' '*26}{result:<10}")
    step += 1
    
    # Procesar resto de bits
    for bit in bits[1:]:
        # Siempre: elevar al cuadrado
        old_result = result
        result = (result * result) % mod
        print(f"{step+1:<6} {'-':<6} result = {old_result}² mod {mod}{' '*16}{result:<10}")
        step += 1
        
        # Si bit=1: multiplicar por base
        if bit:
            old_result = result
            result = (result * base) % mod
            print(f"{step+1:<6} {1:<6} result = {old_result} × {base} mod {mod}{' '*14}{result:<10}")
            step += 1
    
    print("-"*70)
    print(f"\n✓ Resultado final: {base}^{exp} mod {mod} = {result}\n")
    return result

# Ejemplo: Mostrar cómo se calcula 2478^17 mod 3233
print("EJEMPLO: Exponenciación con Trazabilidad")
result = modular_exponentiation_traced(2478, 17, 3233)

---

## Suite de Pruebas Completa

In [ ]:
def encrypt_message(plaintext: str, e: int, n: int) -> list:
    """Cifra un mensaje completo"""
    numeric_str = text_to_numbers(plaintext)
    blocks = split_into_blocks(numeric_str, n)
    encrypted_blocks = []
    for block_value, num_letters in blocks:
        cipher_value = modular_exponentiation(block_value, e, n)
        encrypted_blocks.append((cipher_value, num_letters))
    return encrypted_blocks

def decrypt_message(encrypted_blocks: list, d: int, n: int) -> str:
    """Descifra un mensaje completo"""
    numeric_str = ""
    for cipher_value, num_letters in encrypted_blocks:
        block_value = modular_exponentiation(cipher_value, d, n)
        expected_length = num_letters * 2
        block_str = str(block_value).zfill(expected_length)
        numeric_str += block_str
    return numbers_to_text(numeric_str)

# SUITE DE PRUEBAS
print("="*70)
print("SUITE DE PRUEBAS COMPLETA")
print("="*70)

test_messages = [
    "A",
    "HELLO",
    "ATTACK",
    "CRYPTOGRAPHY",
    "THEQUICKBROWNFOX"
]

print(f"\nUsando claves: e={e}, d={d}, n={n}")
print(f"\n{'Mensaje':<20} {'Cifrado':<30} {'Recuperado':<20} {'OK?'}")
print("-"*80)

all_passed = True

for msg in test_messages:
    encrypted = encrypt_message(msg, e, n)
    cipher_str = str([c for c, _ in encrypted])
    decrypted = decrypt_message(encrypted, d, n)
    
    passed = (msg == decrypted)
    all_passed = all_passed and passed
    status = "✓" if passed else "✗"
    
    print(f"{msg:<20} {cipher_str:<30} {decrypted:<20} {status}")

print("-"*80)
if all_passed:
    print("\n✓✓✓ TODAS LAS PRUEBAS PASARON ✓✓✓")
else:
    print("\n✗✗✗ ALGUNAS PRUEBAS FALLARON ✗✗✗")

---

## 📚 Conclusiones y Conceptos Clave

### ✅ Conceptos Fundamentales

1. **Seguridad de RSA**:
   - Se basa en la dificultad de factorizar n = p×q
   - Con primos grandes (2048+ bits), es computacionalmente imposible

2. **Teorema de Euler**:
   - $(M^e)^d \equiv M \pmod{n}$ garantiza reversibilidad
   - Por eso: cifrar con e y descifrar con d funciona

3. **Exponenciación Rápida**:
   - Sin ella, RSA sería impráctico
   - Reduce O(e) operaciones a O(log e)

4. **Padding**:
   - Crítico para reconstruir mensajes correctamente
   - Preserva ceros a la izquierda: 210 → "000210"

### ⚠️ Limitaciones de Esta Implementación

- **Educativa**: NO usar en producción
- **Primos pequeños**: Fácilmente factorizables
- **Sin padding criptográfico**: No hay OAEP o PKCS#1
- **Sin protección**: Vulnerable a ataques de temporización

### 🔐 Para Uso Real

```python
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

# Generar claves RSA de 2048 bits
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
```

---

## 📖 Referencias

- **Rivest, Shamir, Adleman** (1977): "A Method for Obtaining Digital Signatures and Public-Key Cryptosystems"
- **RFC 8017**: PKCS #1: RSA Cryptography Specifications Version 2.2
- **NIST FIPS 186-5**: Digital Signature Standard (DSS)

---

*Laboratorio completado exitosamente* ✅